# Information Retrieval and Text Analytics Project

## Introduction

**Overview of Information Retrieval (IR) Systems:** An information retrieval system is a system or software designed to search for and retrieve relevant information from a large collection of unstructured or semi-structured data, such as text documents, images, or multimedia content, based on user queries. These systems are essential in handling vast amounts of information, such as those found on the internet, in digital libraries, or within enterprise data systems.

Examples of information retrieval systems:
1.   **Google:** A search engine that retrieves relevant web pages based on keywords entered by the user.
2.   **Amazon:** An e-commerce platform where users search for products by entering keywords or browsing categories.
3.   **PubMed:** A database for medical research articles, allowing users to search for academic papers related to health and science.



**Background:** Data can be found everywhere around us. With the proliferation of digital devices, vast amounts of data are being generated and shared. We can find valuable information in news, books, papers, documentations and wikis. Though to find this valuable information, one must swim through a sea of redundant and sometimes useless data. Therefore, there is a growing need for a way to quickly obtain and sift through this data to extract valuable insights. Many information retrieval models were developed to handle said issue, albeit with varying performances. This creates the issue where we want to know which model performs better in the realm of information retrieval.



**Objective:** This project aims to develop a system that retrieves relevant information from text datasets using 3 different information retrieval models and comparing between their performances to know which one is better. The system also allows us to improve the understanding and organization of the data through text analytics. This includes leveraging preprocessing techniques to enhance text representation, applying robust retrieval methods to ensure accuracy, and utilizing visualizations to provide actionable insights and evaluate performance effectively.

**Scope:** The scope includes implementing advanced preprocessing techniques such as tokenization, case standardization, stopwords removal, stemming, and TF-IDF to improve text representation. Additionally, it encompasses the integration of Vector Space Model, Boolean Retrieval Model, and BM25 retrieval algorithms for ranking and identifying relevant information based on user queries. We compare between the performances of these algorithms. To enhance usability, the project will also incorporate visualization tools such as word clouds for top keywords in documents, Frequency distribution of words, document-query similarity scores (e.g., bar charts), and clustering topics using LDA (Latent Dirichlet Allocation). The project will be limited to textual data and will not cover multimedia or non-textual information retrieval.

## Data Collection

In [31]:
# Downloading required dataset(s)
!pip install kagglehub
!pip install rank_bm25
!pip install nltk
!pip install scikit-learn
import kagglehub
import nltk
import os
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# path = kagglehub.dataset_download("crawford/20-newsgroups")
# The dataset will be download in /home/<user>/.cache/kagglehub


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Data Inspection

In [32]:
# Specify the file path
file_path = "./20-newsgroups/talk.politics.guns.txt"  # Replace with the actual path to your file

# Check if the file exists
if os.path.exists(file_path):
    # Open the file and read its contents
    with open(file_path, 'r', encoding='latin-1') as file:  # Use 'latin-1' encoding
        file_contents = file.read()

    # Print or process the file contents
    print(file_contents[:200])  # Print the first 200 characters of the file contents
else:
    print(f"Error: File not found at '{file_path}'")

Newsgroup: talk.politics.guns
document_id: 53293
From: manes@magpie.linknet.com (Steve Manes)
Subject: Re: Gun Control (was Re: We're Mad as Hell at the TV News)

hambidge@bms.com wrote:
: In article 


# Data Preprocessing

#### Implementing tokenization, lowercasing, stopwords removal, stemming and lemmatization.

In [4]:
def preprocess_text(text):
    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Lowercasing
    tokens = [token.lower() for token in tokens]

    # Stopword Removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and token.isalnum()]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

#### Implementing Vectorization (BoW/TF-IDF)

In [5]:
def vectorize_texts(texts, method='bow'):
    """
    Vectorize texts using Bag of Words (BOW) or TF-IDF.

    Parameters:
    texts (list of str): List of texts to vectorize.
    method (str): Method of vectorization ('bow' or 'tfidf').

    Returns:
    X (sparse matrix): Vectorized text data.
    vectorizer (Vectorizer object): Fitted vectorizer.
    """
    if method == 'bow':
        vectorizer = CountVectorizer()
    elif method == 'tfidf':
        vectorizer = TfidfVectorizer()
    else:
        raise ValueError("Method must be 'bow' or 'tfidf'")

    X = vectorizer.fit_transform(texts)
    return X, vectorizer

##### Read and Process all the documents

In [ ]:
def read_and_preprocess_files(directory_path='./20-newsgroups'):
    preprocessed_texts = []
    filenames = []
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='latin-1') as file:
                text = file.read()
                preprocessed_text = preprocess_text(text)
                preprocessed_texts.append(preprocessed_text)
                filenames.append(filename)
    return preprocessed_texts, filenames

In [ ]:
preprocessed_texts, filenames = read_and_preprocess_files()
preprocessed_texts[:][0][:50]

#### BoW Vectorization

In [50]:
X_bow, vectorizer_bow = vectorize_texts(preprocessed_texts, method='bow')
print(X_bow)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 255181 stored elements and shape (20, 116313)>
  Coords	Values
  (0, 74613)	528
  (0, 100478)	2805
  (0, 52997)	303
  (0, 34556)	2022
  (0, 92161)	57
  (0, 4454)	30
  (0, 46253)	9
  (0, 8041)	24
  (0, 108649)	207
  (0, 31229)	105
  (0, 83714)	162
  (0, 107442)	96
  (0, 54903)	225
  (0, 91836)	1389
  (0, 54679)	51
  (0, 45820)	45
  (0, 53899)	75
  (0, 39308)	6
  (0, 99943)	9
  (0, 34411)	21
  (0, 85035)	6
  (0, 34930)	93
  (0, 111995)	3015
  (0, 53531)	6
  (0, 38426)	99
  :	:
  (19, 55777)	2
  (19, 60621)	2
  (19, 104123)	4
  (19, 66362)	2
  (19, 47850)	2
  (19, 30998)	2
  (19, 63349)	2
  (19, 112001)	2
  (19, 27868)	2
  (19, 27869)	2
  (19, 27870)	2
  (19, 76776)	2
  (19, 103323)	2
  (19, 27871)	2
  (19, 35716)	2
  (19, 27872)	2
  (19, 9345)	2
  (19, 27873)	2
  (19, 89845)	2
  (19, 74537)	2
  (19, 63808)	2
  (19, 29204)	2
  (19, 27076)	2
  (19, 27874)	2
  (19, 36800)	2


#### TF-IDF Vectorization

In [51]:
X_tfidf, vectorizer_tfidf = vectorize_texts(preprocessed_texts, method='tfidf')
print(X_tfidf)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 255181 stored elements and shape (20, 116313)>
  Coords	Values
  (0, 74613)	0.056268502858254944
  (0, 100478)	0.17160572527992948
  (0, 52997)	0.018537089040933562
  (0, 34556)	0.228516196021637
  (0, 92161)	0.0034871751661162144
  (0, 4454)	0.0018353553505874812
  (0, 46253)	0.0006057132185266019
  (0, 8041)	0.001468284280469985
  (0, 108649)	0.012663951919053621
  (0, 31229)	0.006423743727056184
  (0, 83714)	0.009910918893172398
  (0, 107442)	0.00587313712187994
  (0, 54903)	0.01588707469203853
  (0, 91836)	0.10293335559664273
  (0, 54679)	0.003432374904984077
  (0, 45820)	0.006201930851749967
  (0, 53899)	0.0050476101543883494
  (0, 39308)	0.0006394148052074424
  (0, 99943)	0.0007358704411183083
  (0, 34411)	0.0018056695343612325
  (0, 85035)	0.0009757586246952326
  (0, 34930)	0.005689601586821192
  (0, 111995)	0.18445321273404186
  (0, 53531)	0.001230192902012297
  (0, 38426)	0.006056672656938688
  :	:
  (19, 55777)	0.0

# Retrieval Models

##### Set the query

In [52]:
query = "god will punish you"

## Implementing BM25 (Best Matching 25)

In [53]:
def apply_bm25(query=query, preprocessed_texts=preprocessed_texts, filenames=filenames):
    """
    Apply BM25 algorithm to retrieve and rank documents based on a query.
    Parameters:
    query (str): The search query.
    preprocessed_texts (list of str): List of preprocessed texts.
    filenames (list of str): List of filenames corresponding to the texts.
    Returns:
    list: List of retrieved document filenames.
    dict: Dictionary of filenames and their BM25 scores.
    """
    # Tokenize the preprocessed texts
    tokenized_texts = [text.split() for text in preprocessed_texts]

    # Initialize BM25 with the tokenized corpus
    bm25 = BM25Okapi(tokenized_texts)

    # Preprocess and tokenize the query
    tokenized_query = preprocess_text(query).split()

    # Get BM25 scores for the query
    scores = bm25.get_scores(tokenized_query)

    # Rank documents based on the scores
    top_n = 5
    top_n_indices = scores.argsort()[-top_n:][::-1]

    # Retrieve the filenames of the top-ranked documents
    retrieved_docs = [filenames[i] for i in top_n_indices]

    # Create a dictionary of filenames and their scores
    ranked_documents = {filenames[i]: scores[i] for i in top_n_indices}

    return retrieved_docs, ranked_documents

In [54]:
bm25_retrieved_docs, bm25_ranked_documents = apply_bm25(query=query)

In [55]:
bm25_ranked_documents

{'alt.atheism.txt': np.float64(2.6840983264916263),
 'soc.religion.christian.txt': np.float64(2.682816014354655),
 'talk.religion.misc.txt': np.float64(2.6755093572955917),
 'talk.politics.mideast.txt': np.float64(2.6526685421754683),
 'misc.forsale.txt': np.float64(2.634362546834729)}

## Implementing Vector Space Model (VSM)

In [56]:
def apply_vsm(query=query, preprocessed_texts=preprocessed_texts, vectorizer=vectorizer_bow):
    # Vectorize the preprocessed texts
    X = vectorizer.fit_transform(preprocessed_texts)

    # Vectorize the query
    query_vector = vectorizer.transform([query])

    # Calculate cosine similarity between the query and the documents
    similarities = cosine_similarity(query_vector, X).flatten()

    # Rank documents based on similarity scores
    ranked_indices = similarities.argsort()[::-1]
    ranked_indices = ranked_indices[:5]

    # Print the ranked documents with filenames and similarity scores
    dict = {}
    for i in ranked_indices[:]:  # Display top 20 results
        dict[filenames[i]] = similarities[i]
        # print(f"Document: {filenames[i]}, Similarity: {similarities[i]}")

    retrieved_documents = [filenames[i] for i in ranked_indices]
    return retrieved_documents, dict

In [57]:
vsm_bow_retrieved_documents, vsm_bow_ranked_documents = apply_vsm(vectorizer=vectorizer_bow)

In [58]:
vsm_bow_ranked_documents 

{'soc.religion.christian.txt': np.float64(0.19818620441718507),
 'talk.religion.misc.txt': np.float64(0.1513463575635367),
 'alt.atheism.txt': np.float64(0.13592968621712972),
 'misc.forsale.txt': np.float64(0.021518098056080452),
 'talk.politics.mideast.txt': np.float64(0.014887751854709957)}

In [59]:
vsm_tfidf_retrieved_documents, vsm_tfidf_ranked_documents = apply_vsm(vectorizer=vectorizer_tfidf)

In [60]:
vsm_tfidf_ranked_documents

{'soc.religion.christian.txt': np.float64(0.09720759248313056),
 'talk.religion.misc.txt': np.float64(0.0731307112782049),
 'alt.atheism.txt': np.float64(0.0668824705021276),
 'misc.forsale.txt': np.float64(0.013619322244066104),
 'talk.politics.misc.txt': np.float64(0.007623357613257748)}

# Evaluation

#### Decide the relevant documents

In [63]:
def get_documents_names(directory_path='./20-newsgroups'):
    # Create a dictionary of filenames with keys from 1 to 20
    filenames_dict = {i+1: filename for i, filename in enumerate(os.listdir(directory_path)[:20])}

    # Print the dictionary for user reference
    for key, value in filenames_dict.items():
        print(f"{key}: {value}")

    return filenames_dict

filenames_dict = get_documents_names()

1: alt.atheism.txt
2: comp.graphics.txt
3: comp.os.ms-windows.misc.txt
4: comp.sys.ibm.pc.hardware.txt
5: comp.sys.mac.hardware.txt
6: comp.windows.x.txt
7: misc.forsale.txt
8: rec.autos.txt
9: rec.motorcycles.txt
10: rec.sport.baseball.txt
11: rec.sport.hockey.txt
12: sci.crypt.txt
13: sci.electronics.txt
14: sci.med.txt
15: sci.space.txt
16: soc.religion.christian.txt
17: talk.politics.guns.txt
18: talk.politics.mideast.txt
19: talk.politics.misc.txt
20: talk.religion.misc.txt


In [68]:
def get_relevant_documents(filenames_dict):
    # Get user input for relevant document numbers
    relevant_numbers = input("Enter the numbers related to the relevant documents, separated by commas: ")
    relevant_numbers = [int(num.strip()) for num in relevant_numbers.split(',')]

    # Save the corresponding filenames in a list
    relevant_docs = [filenames_dict[num] for num in relevant_numbers if num in filenames_dict]

    return relevant_docs

# Example usage
relevant_docs = get_relevant_documents(filenames_dict)
print(f"Relevant documents: {relevant_docs}")

Relevant documents: ['alt.atheism.txt', 'soc.religion.christian.txt', 'talk.religion.misc.txt', 'comp.sys.ibm.pc.hardware.txt']


### Precesion

In [74]:
def calculate_precision(retrieved_docs, relevant_docs):
    """
    Calculate precision.
    {key: value, key: value}
    Parameters:
    retrieved_docs (list): List of retrieved document filenames.
    relevant_docs (list): List of relevant document filenames.

    Returns:
    float: Precision value.
    """
    # Convert lists to sets for easier calculation
    retrieved_set = set(retrieved_docs)
    relevant_set = set(relevant_docs)

    # Calculate the number of relevant documents retrieved
    relevant_retrieved = retrieved_set.intersection(relevant_set)

    # Calculate precision
    precision = len(relevant_retrieved) / len(retrieved_set) if retrieved_set else 0

    return precision

In [75]:
# Calculate precision for BM25
bm25_precision = calculate_precision(bm25_retrieved_docs, relevant_docs)
# Calculate precision for VSM with BoW
vsm_bow_precision = calculate_precision(vsm_bow_retrieved_documents, relevant_docs)
# Calculate precision for VSM with TF-IDF
vsm_tfidf_precision = calculate_precision(vsm_tfidf_retrieved_documents, relevant_docs)

print(f"BM25 Precision: {bm25_precision}")
print(f"VSM with BoW Precision: {vsm_bow_precision}")
print(f"VSM with TF-IDF Precision: {vsm_tfidf_precision}")

BM25 Precision: 0.6
VSM with BoW Precision: 0.6
VSM with TF-IDF Precision: 0.6


### Recall

In [76]:
def calculate_recall(retrieved_docs, relevant_docs):
    """
    Calculate recall.

    Parameters:
    retrieved_docs (list): List of retrieved document filenames.
    relevant_docs (list): List of relevant document filenames.

    Returns:
    float: Recall value.
    """
    # Convert lists to sets for easier calculation
    retrieved_set = set(retrieved_docs)
    relevant_set = set(relevant_docs)

    # Calculate the number of relevant documents retrieved
    relevant_retrieved = retrieved_set.intersection(relevant_set)

    # Calculate recall
    recall = len(relevant_retrieved) / len(relevant_set) if relevant_set else 0

    return recall

In [77]:
# Calculate recall for BM25
bm25_recall = calculate_recall(bm25_retrieved_docs, relevant_docs)
# Calculate recall for VSM with BoW
vsm_bow_recall = calculate_recall(vsm_bow_retrieved_documents, relevant_docs)
# Calculate recall for VSM with TF-IDF
vsm_tfidf_recall = calculate_recall(vsm_tfidf_retrieved_documents, relevant_docs)

print(f"BM25 Recall: {bm25_recall}")
print(f"VSM with BoW Recall: {vsm_bow_recall}")
print(f"VSM with TF-IDF Recall: {vsm_tfidf_recall}")

BM25 Recall: 0.75
VSM with BoW Recall: 0.75
VSM with TF-IDF Recall: 0.75


### Mean Average Precision (MAP)

In [78]:
def calculate_mean_average_precision(retrieved_docs_list, relevant_docs_list):
    """
    Calculate Mean Average Precision (MAP).

    Parameters:
    retrieved_docs_list (list of lists): List of retrieved document filenames for each query.
    relevant_docs_list (list of lists): List of relevant document filenames for each query.

    Returns:
    float: Mean Average Precision value.
    """
    def calculate_average_precision(retrieved_docs, relevant_docs):
        """
        Calculate the average precision for a single query.


        Parameters:
        retrieved_docs (list): List of retrieved document filenames.
        relevant_docs (list): List of relevant document filenames.

        Returns:
        float: Average precision value.
        """
        retrieved_set = set(retrieved_docs)
        relevant_set = set(relevant_docs)
        
        if not relevant_set:
            return 0.0

        relevant_retrieved = 0
        precision_sum = 0.0

        for i, doc in enumerate(retrieved_set):
            if doc in relevant_set:
                relevant_retrieved += 1
                precision_sum += relevant_retrieved / (i + 1)

        return precision_sum / len(relevant_set)

    average_precisions = [
        calculate_average_precision(retrieved_docs, relevant_docs)
        for retrieved_docs, relevant_docs in zip(retrieved_docs_list, relevant_docs_list)
    ]

    return sum(average_precisions) / len(average_precisions) if average_precisions else 0.0

In [79]:
# Calculate Mean Average Precision
retrieved_docs_list = [bm25_retrieved_docs, vsm_bow_retrieved_documents, vsm_tfidf_retrieved_documents]
relevant_docs_list = [relevant_docs] * 3
mean_average_precision = calculate_mean_average_precision(retrieved_docs_list, relevant_docs_list)
print(f"Mean Average Precision: {mean_average_precision}")

Mean Average Precision: 0.4833333333333334


# Visualization

In [80]:
def count_word_in_files(directory_path='./20-newsgroups', word="god will be with you"):
    """
    Count the occurrences of a word in all files within a directory.

    Parameters:
    directory_path (str): Path to the directory containing text files.
    word (str): Word to count occurrences of.

    Returns:
    dict: A dictionary with filenames as keys and word counts as values, sorted by word counts.
    """
    word_count = {}
    word = word.lower()  # Convert the word to lowercase for case-insensitive matching

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='latin-1') as file:
                text = file.read()

                # Preprocess the text: remove punctuation, convert to lowercase
                text = re.sub(r'[^\w\s]', '', text)
                text = text.lower()

                # Count occurrences of the word
                count = text.split().count(word)
                word_count[filename] = count

    # Sort the dictionary by word counts in descending order
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1], reverse=True))

    return sorted_word_count

word_counts = count_word_in_files()
print(word_counts)

{'alt.atheism.txt': 0, 'comp.graphics.txt': 0, 'comp.os.ms-windows.misc.txt': 0, 'comp.sys.ibm.pc.hardware.txt': 0, 'comp.sys.mac.hardware.txt': 0, 'comp.windows.x.txt': 0, 'misc.forsale.txt': 0, 'rec.autos.txt': 0, 'rec.motorcycles.txt': 0, 'rec.sport.baseball.txt': 0, 'rec.sport.hockey.txt': 0, 'sci.crypt.txt': 0, 'sci.electronics.txt': 0, 'sci.med.txt': 0, 'sci.space.txt': 0, 'soc.religion.christian.txt': 0, 'talk.politics.guns.txt': 0, 'talk.politics.mideast.txt': 0, 'talk.politics.misc.txt': 0, 'talk.religion.misc.txt': 0}
